In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from hankel import HankelTransform

In [2]:
# We need to import spline interpolation for the inverse transformation to get smoothness
from scipy.interpolate import InterpolatedUnivariateSpline as spline

## Chen and Oldenburg 2004 implementation of the 2 layer system

In [6]:
# Electrode is placed at the interface between rho1 and rho2
class halfspace_2_layer:
    def __init__(self, rho1, rho2, h_overburden, tx_current):
        """
        Returns the field magnitude of a buried electrode located at the interface between media
        rho1 and rho2, thickness of rho1 being defined with h_overburden
        @return: B_field magnitude
        """
        # Define a hankel transform of the first order and store it in the object
        self.hankelTF = HankelTransform(
            nu= 1,     # The order of the bessel function
            N = 120,   # Number of steps in the integration
            h = 0.001   # Proxy for "size" of steps in integration
        )
        # We'll have to tweak these parameters to optimize for 
        # speed and accuracy in the indefinite integral used in the defn of the Hankel Transform
        
        self.rho1 = rho1
        self.rho2 = rho2
        self.h_overburden = h_overburden
        
        self.tx_current = tx_current
    
    def B_Field(self, lbda, depth):
        # Switch statement
        if depth > self.h_overburden:
            return self.lower_layer_bfield(lbda, depth)
        else:
            return self.upper_layer_bfield(lbda, depth)
    
    # BField in the upper layer
    def upper_layer_bfield(self, lbda, depth):
        # Equation 15 from Chen and Oldenburg (2004)
        # len(lbda) must be len(depth)
        D1 = self._D_1(lbda)
        U1 = self._U_1(lbda)
        hnkl_depth = self.hankelTF.transform(lambda x: depth, lbda, ret_err=False)
        
        term1 = self.tx_current / 2 / np.pi / lbda
        term2 = D1 * np.exp(-lbda * hnkl_depth)
        term3 = U1 * np.exp(lbda * hnkl_depth)
        return term1 + term2 + term3
    
    def lower_layer_bfield(self, lbda, depth):
        # Equation 16 from Chen and Oldenburg (2004)
        D2 = self._D_2(lbda)
        U2 = self._U_2(lbda)
        hnkl_depth = self.hankelTF.transform(lambda x: depth - self.h_overburden, lbda, ret_err=False)
        
        term1 = D2 * np.exp(-lbda * hnkl_depth)
        term2 = U2 * np.exp(lbda * hnkl_depth)
        return term1 + term2
    
    ### LAYER COEFFICIENTS ###
    def _D_1(self, lbda):
        # Equation 17 from Chen and Oldenburg (2004)
        hnkl_h1 = self.hankelTF.transform(lambda x: self.h_overburden, lbda, ret_err=False)
                                
        bot = (1-np.exp(-2*lbda*hnkl_h1)) + self.rho2 / self.rho1 * (1+np.exp(-2*lbda*hnkl_h1))
        return self.tx_current * np.exp(-lbda * hnkl_h1) / 2 / np.pi / lbda / bot
    
    def _U_1(self, lbda):
        return -self._D_1(lbda)
    
    def _D_2(self, lbda):
        # Equation 18 from Chen and Oldenburg (2004)
        hnkl_h1 = self.hankelTF.transform(lambda x: self.h_overburden, lbda, ret_err=False)
        top = (1 + np.exp(-2 * lbda * hnkl_h1))
        bot = (1 - np.exp(-2 * lbda * hnkl_h1)) + self.rho2 / self.rho1 * (1 + np.exp(-2*lbda*hnkl_h1))
        return self.tx_current * top * self.rho2 / self.rho1 / 2 / np.pi / lbda / bot
    
    def _U_2(self, lbda):
        return 0


In [7]:
# Initialize a
earth = halfspace_2_layer(
                  rho1=10,
                  rho2=20,
                  h_overburden=250,
                  tx_current=2)

In [8]:
# We choose a lambda value for the hankel transformation
lbda = 25

# Depth Domain in Realspace (m)
dep = np.linspace(0, 500, 500)[1:]

# Domain transformed into hankel space for plotting (This isn't used because the class above will
# do the transformation locally
d_hankelspace = list(map(lambda d: earth.hankelTF.transform(lambda x: d, lbda, ret_err=False), dep))

# Compute the field in hankel space
hankel_field = np.array(list(map(lambda d: earth.B_Field(lbda, d), dep)))

interp_hankel_field = spline(d_hankelspace, hankel_field) #  Interpolate the computed field
real_field = earth.hankelTF.transform(interp_hankel_field, dep, inverse=True, ret_err=False)

hankelfieldplot = go.Scatter(x=hankel_field, y=d_hankelspace, name='Hankel Space')
realfieldplot = go.Scatter(x=real_field, y=dep, name='Real Space (ERROR)')

figure = make_subplots(rows=1, cols=2)
figure.update_layout(title_text="Hankel Space B-Field")
figure.add_trace(hankelfieldplot, row=1, col=1)
figure.add_trace(realfieldplot, row=1, col=2)

figure.update_xaxes(title_text="Azimuthal B-Field (T)", row=1, col=1)
figure.update_xaxes(title_text="Azimuthal B-Field (T)", row=1, col=2)
figure.update_yaxes(title_text="Hankel Space Depth", autorange="reversed", row=1, col=1)
figure.update_yaxes(title_text="Real Space Depth", autorange="reversed", row=1, col=2)
figure.show()

ValueError: x must be increasing if s > 0